In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.sparse import csr_matrix, lil_matrix
from IPython.display import HTML
plt.rcParams['animation.html'] = 'jshtml'
# plt.rcParams['animation.embed_limit'] = 1024 

In [2]:
def simulate_cahn_hilliard():
    Nx, Ny = 128, 128
    dx = dy = 1
    dt = 0.01
    gamma = np.sqrt(3)
    total_time = 50000
    x = np.linspace(-0.5, 0.5, Nx)
    y = np.linspace(0, 1, Ny)
    X, Y = np.meshgrid(x, y)
    def build_fem_matrices():
        M = lil_matrix((Nx*Ny, Nx*Ny))
        K = lil_matrix((Nx*Ny, Nx*Ny))
        for i in range(Nx):
            for j in range(Ny):
                idx = i + j*Nx
                M[idx, idx] = dx * dy
                K[idx, idx] = 4 / (dx * dy)
                if i > 0:
                    K[idx, idx-1] = -1 / (dx * dy)
                if i < Nx-1:
                    K[idx, idx+1] = -1 / (dx * dy)
                if j > 0:
                    K[idx, idx-Nx] = -1 / (dx * dy)
                if j < Ny-1:
                    K[idx, idx+Nx] = -1 / (dx * dy)
        return csr_matrix(M), csr_matrix(K)
    M, K = build_fem_matrices()
    M_inv_diag = 1 / M.diagonal()
    M_inv = csr_matrix(np.diag(M_inv_diag))
    np.random.seed(42)
    u = np.clip(np.random.uniform(-1, 1, (Nx, Ny)), -1, 1)

    def laplacian_fem(f):
        f_vec = f.flatten()
        lap = -M_inv.dot(K.dot(f_vec)) 
        return lap.reshape((Nx, Ny))

    def chemical_potential(u):
        return u**3 - u - gamma**2 * laplacian_fem(u)

    def mobility(u):
        return 1 - u**2

    frames = []
    times = []
    for t in tqdm(range(total_time)):
        mu = chemical_potential(u)
        M_u = np.clip(mobility(u), 0, 1)
        u += dt * laplacian_fem(M_u * mu)
        if t % 30 == 0:
            frames.append(u.copy())
            times.append(t * dt)
    return frames, times

In [3]:
frames, times = simulate_cahn_hilliard()

100%|██████████| 50000/50000 [02:09<00:00, 385.64it/s]


In [ ]:
colors = [(0, 0, 0), (1, 0, 0), (1, 1, 0)]
cmap = LinearSegmentedColormap.from_list("custom_colormap", colors, N=256)

fig, ax = plt.subplots()
im = ax.imshow(frames[0], cmap=cmap, extent=(-0.5, 0.5, 0, 1), vmin=-1, vmax=1)
plt.colorbar(im)

time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color="white")


def animate(i):
    im.set_array(frames[i])
    time_text.set_text(f'Temps: {times[i]/100:.3f} s')
    return [im, time_text]

anim = FuncAnimation(fig, animate, frames=len(frames), interval=50, blit=True)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
np.save("fem_canh_hilliard.npy", frames)